In [9]:
import importlib

import training
import utils
import training_constants as tc
from preprocessing import encode_labels, load_data, split_train_test_data

In [10]:
importlib.reload(training)
importlib.reload(utils)

<module 'utils' from '/Users/patricia/Documents/code/python/behavior-detection/scripts/utils.py'>

In [11]:
X, y = load_data()
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5525 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.2+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 5525 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5525 non-null   object
dtypes: object(1)
memory usage: 43.3+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [12]:
X_train, X_test, y_train, y_test = split_train_test_data(X, y)


Tamanho do conjunto de treino: 3867
Tamanho do conjunto de teste: 1658


In [13]:
y_train, label_encoder = encode_labels(y_train)
y_test = label_encoder.transform(y_test)


In [14]:
trained_models = training.train_model(X_train, y_train, tc.CROSS_VALIDATION)


Training and evaluating Decision Tree with Cross-Validation:
Cross-Validation Best Result for Decision Tree: 0.712425452177685

Training and evaluating Random Forest with Cross-Validation:
Cross-Validation Best Result for Random Forest: 0.7516288886805983

Training and evaluating Gradient Boosting with Cross-Validation:
Cross-Validation Best Result for Gradient Boosting: 0.6754304409234583

Training and evaluating XGBoost with Cross-Validation:
Cross-Validation Best Result for XGBoost: 0.7009606054331514


In [19]:
trained_models = results

In [21]:
import evaluation
import importlib

importlib.reload(evaluation)
dirpath = "/Users/patricia/Documents/code/python/behavior-detection/output/"

reports = evaluation.generate_reports(trained_models, X_train, y_train, X_test, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)


Evaluating Decision Tree:

Training set report:
Class 0 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 17.0
Class 1 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 434.0
Class 2 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 637.0
Class 3 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 2231.0
Class 4 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 285.0
Class 5 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 263.0
Accuracy - F1-Score: 1.0
Class macro avg - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 3867.0
Class weighted avg - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 3867.0
Balanced Accuracy: 1.0

Test set report:
Class 0 - Precision: 0.92, Recall: 0.58, F1-Score: 0.71, Support: 19.0
Class 1 - Precision: 0.7, Recall: 0.68, F1-Score: 0.69, Support: 195.0
Class 2 - Precision: 0.76, Recall: 0.77, F1-Score: 0.77, Support: 270.0
Class 3 - Precision: 0.87, Recall: 0.87, F1-Score: 0.87, Support: 928.0
Class 4 - Precision: 0.86,